# License 
***
Copyright (C) 2017--2022 J. Patrick Hall, jphall@gwu.edu

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

## Installs and Imports

1. Standard Python imports and config

In [ ]:
import pandas as pd # import pandas for easy data manipulation using data frames

from matplotlib import pyplot as plt # plotting
import numpy as np                   # basic array and matric handling

# to upload local files
import io
from google.colab import files  

SEED = 12345 # seed for better reproducibility

# display matplotlib graphics in notebook
%matplotlib inline

2. Install Java

In [ ]:
# install Java for h2o backend
!apt-get install default-jre
!java -version

3. Install h2o

In [ ]:
# install h2o
!pip install h2o 

4. Import h2o package and required classes

In [ ]:
# import h2o and required classes
import h2o
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.grid.grid_search import H2OGridSearch

5. Start h2o Server

In [ ]:
# start and connect to h2o server
h2o.init()

## Load example data

6. Upload class example data

In [ ]:
# special google collab command to upload a file from computer
uploaded = files.upload() # REQUIRES STUDENT INPUT

In [ ]:
# 7
uploaded.keys() # what is stored in that Python object?

8. Covert to Pandas DataFrame

In [ ]:
# convert data to Pandas DataFrame
raw = pd.read_csv(io.StringIO(uploaded['loan_clean.csv'].decode('utf-8'))) # name in quotes here must match name above

9. Assign basic metadata (measurement level) for h2o

In [ ]:
# define input feature measurement levels 
# strings automatically parsed as enums (nominal/categorical)
# numbers automatically parsed as numeric
col_types = {'bad_loan': 'enum'}

10. Covert to h2o frame

In [ ]:
hraw = h2o.H2OFrame(raw, column_types=col_types)

11. Split into train, validation, and test partitions

In [ ]:
# split into 40% training, 30% validation, and 30% test
train, valid, test = hraw.split_frame([0.4, 0.3])

12. Assign basic metadata (modeling roles) for h2o

In [ ]:
# assign target and inputs
y = 'bad_loan'
X = [name for name in train.columns if name not in ['id', '_WARN_', 'GRP_home_ownership', y]]
print(y)
print(X)

13. Double-check that h2o treats y as categorical

In [ ]:
# set target to catgorical - for binary classification
train[y] = train[y].asfactor()
valid[y] = valid[y].asfactor()
test[y] = test[y].asfactor()

## Train ANN using random grid search

14. Conduct random grid search

In [ ]:
# train ANN with random hyperparameter search
# train many different ANN models with random hyperparameters
# and select best model based on validation error

# define random grid search parameters
hyper_parameters = {'hidden':[], # REQUIRES STUDENT INPUT
                    'l1':[s/1e4 for s in range(0, 1000, 100)],
                    'l2':[s/1e5 for s in range(0, 1000, 100)],
                    'input_dropout_ratio':[s/1e2 for s in range(0, 20, 2)]}

# define search strategy
search_criteria = {'strategy':'', # REQUIRES STUDENT INPUT -- random search!
                   'max_models': 20,
                   'max_runtime_secs': 1200,
                   'seed': } # REQUIRES STUDENT INPUT

# initialize grid search
gsearch = H2OGridSearch(H2ODeepLearningEstimator,
                        hyper_params=hyper_parameters,
                        search_criteria=search_criteria)

# execute training w/ grid search
gsearch.train(x=X,
              y=y,
              training_frame=train,
              validation_frame=valid,
              reproducible=True, # force SGD to reproducible, but slow
              seed=) # REQUIRES STUDENT INPUT

15. Display grid search results

In [ ]:
# show grid search results
gsearch.show()

# select best model
best_ann = gsearch.get_grid()[0]

# print model information
best_ann

16. Display AUC across partitions

In [ ]:
# measure ann AUC
print('Training AUC:',best_ann.auc(train=True))
print('Validation AUC:', best_ann.auc(valid=True))
print('Test AUC:', best_ann.model_performance(test_data=test).auc())

17. Display iteration plot for best model

In [ ]:
_ = best_ann.plot()

18. Score new row of data

In [ ]:
# predict on new data -- REQUIRES STUDENT INPUT
new_row = pd.DataFrame.from_dict({
  "GRP_REP_home_ownership": [],
  "GRP_addr_state": [],
  "GRP_purpose": [],
  "GRP_verification_status": [],
  "STD_IMP_REP_annual_inc": [],
  "STD_IMP_REP_delinq_2yrs": [],
  "STD_IMP_REP_dti": [],
  "STD_IMP_REP_emp_length": [],
  "STD_IMP_REP_int_rate": [],
  "STD_IMP_REP_loan_amnt": [],
  "STD_IMP_REP_longest_credit_lengt": [],
  "STD_IMP_REP_revol_util": [],
  "STD_IMP_REP_term_length":[],
  "STD_IMP_REP_total_acc": []},
  orient='columns')

# generate prediction -- REQUIRES STUDENT INPUT
best_ann.predict(h2o.H2OFrame())

19. Shutdown h2o

In [ ]:
# shutdown h2o
h2o.cluster().shutdown(prompt=False)